In [ ]:
pip install pandas-profiling

In [ ]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

# Prueba Lulo Prueba técnica Lulo Bank - Analytics engineers

Importar módulos

In [2]:
import pandas as pd
import requests
import datetime
import os
from pandas_profiling import ProfileReport
import json
import sqlite3
from sqlalchemy import create_engine
import numpy as np
import ast
import matplotlib.pylab as plt

Creamos una lista con las fechas diarias de diciembre del 2020

In [3]:
start = datetime.date(2020,12,1)
periods = 31
list_dates = []
for day in range(periods):
  date = (start + datetime.timedelta(days = day)).isoformat()
  list_dates.append(date)
print(list_dates)

['2020-12-01', '2020-12-02', '2020-12-03', '2020-12-04', '2020-12-05', '2020-12-06', '2020-12-07', '2020-12-08', '2020-12-09', '2020-12-10', '2020-12-11', '2020-12-12', '2020-12-13', '2020-12-14', '2020-12-15', '2020-12-16', '2020-12-17', '2020-12-18', '2020-12-19', '2020-12-20', '2020-12-21', '2020-12-22', '2020-12-23', '2020-12-24', '2020-12-25', '2020-12-26', '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31']


Indicamos URL del API y en un for recorremos la lista de fechas, que son los parámetros para realizar cada consulta al API de TV maze con el método GET

In [4]:
url = 'http://api.tvmaze.com/schedule/web'

In [5]:
os.getcwd()
os.mkdir("json")
os.mkdir("profiling")

In [ ]:
parametros = {}
list_jsons = []
for i in range(len(list_dates)):
    parametros = {'date':list_dates[i]}
    print(parametros)
    response = requests.get(url, params = parametros)
    print(response)
    json_data = response.json()
    with open('./json/json_data_'+str(i)+'.json', 'w') as json_file:
      json.dump(json_data, json_file)    
    list_jsons.append(pd.json_normalize(response.json()))

Hemos guardado los 31 archivos .json en la carpeta json con la siguiente nomenclatura json_data_i.
Dado que el responde de la consulta al API arrojó json's con la misma estructura pero de cada día de diciembre del 2020, se decide crear un solo dataframe con la lista de los json's y a este dataframe vamos a realizar el análisis y profiling.

In [7]:
df = pd.concat(list_jsons)
df.head()

,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,...,_embedded.show.network.country.code,_embedded.show.network.country.timezone,_embedded.show.network.officialSite,_embedded.show._links.nextepisode.href,_embedded.show.webChannel,_embedded.show.image,_embedded.show.webChannel.country,_embedded.show.dvdCountry.name,_embedded.show.dvdCountry.code,_embedded.show.dvdCountry.timezone
0,1979824,https://www.tvmaze.com/episodes/1979824/sim-fo...,Chanyeol's Episode 16,4,16.0,regular,2020-12-01,06:00,2020-11-30T21:00:00+00:00,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,...,RU,Asia/Kamchatka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8.0,...,NaN,NaN,NaN,https://api.tvmaze.com/episodes/2309427,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3037 entries, 0 to 129
Data columns (total 62 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   id                                          3037 non-null   int64  
 1   url                                         3037 non-null   object 
 2   name                                        3037 non-null   object 
 3   season                                      3037 non-null   int64  
 4   number                                      2956 non-null   float64
 5   type                                        3037 non-null   object 
 6   airdate                                     3037 non-null   object 
 7   airtime                                     3037 non-null   object 
 8   airstamp                                    3037 non-null   object 
 9   runtime                                     2810 non-null   float64
 10  image        

In [9]:
df.describe()

,id,season,number,runtime,image,rating.average,_embedded.show.id,_embedded.show.runtime,_embedded.show.averageRuntime,_embedded.show.rating.average,_embedded.show.weight,_embedded.show.network,_embedded.show.webChannel.id,_embedded.show.externals.thetvdb,_embedded.show.updated,_embedded.show.network.id,_embedded.show.webChannel,_embedded.show.image,_embedded.show.webChannel.country
count,3.037000e+03,3037.000000,2956.000000,2810.000000,0.0,301.000000,3037.000000,2039.000000,2877.000000,423.000000,3037.000000,0.0,2957.000000,2116.000000,3.037000e+03,206.000000,0.0,0.0,0.0
mean,2.029997e+06,149.383602,23.325101,37.760854,NaN,7.943522,46736.111623,38.814615,36.957942,6.848227,37.334541,NaN,145.922895,358840.099716,1.637348e+09,492.616505,NaN,NaN,NaN
std,9.197163e+04,524.145917,51.795918,30.120851,NaN,1.024792,12371.671966,30.807388,28.164194,0.944322,26.772907,NaN,140.005937,56143.543336,1.775756e+07,533.108028,NaN,NaN,NaN
min,1.732625e+06,1.000000,1.000000,1.000000,NaN,3.700000,802.000000,1.000000,1.000000,3.600000,0.000000,NaN,1.000000,73246.000000,1.602172e+09,8.000000,NaN,NaN,NaN
25%,1.978095e+06,1.000000,4.000000,19.000000,NaN,7.300000,44060.000000,20.000000,18.000000,6.600000,16.000000,NaN,21.000000,344750.000000,1.618077e+09,112.000000,NaN,NaN,NaN
50%,1.989343e+06,1.000000,8.000000,32.000000,NaN,7.900000,51915.000000,35.000000,30.000000,7.100000,31.000000,NaN,104.000000,381131.000000,1.644762e+09,308.000000,NaN,NaN,NaN
75%,2.036955e+06,3.000000,20.000000,45.000000,NaN,8.700000,52784.000000,45.000000,45.000000,7.500000,55.000000,NaN,238.000000,392410.000000,1.653583e+09,551.000000,NaN,NaN,NaN
max,2.357150e+06,2020.000000,358.000000,335.000000,NaN,10.000000,62901.000000,300.000000,300.000000,8.800000,100.000000,NaN,533.000000,419045.000000,1.657832e+09,1862.000000,NaN,NaN,NaN


In [10]:
prof = ProfileReport(df)
prof.to_file(output_file='./profiling/prof_result.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas_profiling/model/correlations.py:61: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  (include the error message: '{error}')"""


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prof

Procedemos a la limpieza del dataframe.
1. Cada columna que sea totalmente nula (todos los registros nulos) se elimina del dataframe
2. Las columnas de tipo float con nulos se van a reemplazar por 0.
3. Se van a transformar los nombres de las columnas, haciéndolos más cortos, con el fin de reemplazar palabras inncesarias y facilitar la lecctura de cada campo.
4. Dado que la columna _embedded.show.genres contiene una lista de géneros se van a pasar los valores a filas en la misma columna.

Estas son las columnas que serán descartadas del dataframe:

In [11]:
df_null_columns = df.isna().sum().to_frame()
df_null_columns.rename(columns = {0:'num_nulos'}, inplace=True)
df_null_columns = df_null_columns[df_null_columns['num_nulos'] == len(df.index)]
df_null_columns

,num_nulos
image,3037
_embedded.show.network,3037
_embedded.show.dvdCountry,3037
_embedded.show.webChannel,3037
_embedded.show.image,3037
_embedded.show.webChannel.country,3037


In [12]:
df = df.dropna(axis='columns', how ='all')
df.head()

,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,...,_embedded.show.network.id,_embedded.show.network.name,_embedded.show.network.country.name,_embedded.show.network.country.code,_embedded.show.network.country.timezone,_embedded.show.network.officialSite,_embedded.show._links.nextepisode.href,_embedded.show.dvdCountry.name,_embedded.show.dvdCountry.code,_embedded.show.dvdCountry.timezone
0,1979824,https://www.tvmaze.com/episodes/1979824/sim-fo...,Chanyeol's Episode 16,4,16.0,regular,2020-12-01,06:00,2020-11-30T21:00:00+00:00,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,...,308.0,ТНТ,Russian Federation,RU,Asia/Kamchatka,NaN,NaN,NaN,NaN,NaN
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,https://api.tvmaze.com/episodes/2309427,NaN,NaN,NaN


Ahora el dataframe no tiene estas columnas y procedemos a modificar los nombres de las columnas.

In [13]:
df.columns = df.columns.str.replace("_embedded.", "")
df.columns = df.columns.str.replace(".", "_")
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,...,show_network_id,show_network_name,show_network_country_name,show_network_country_code,show_network_country_timezone,show_network_officialSite,show__links_nextepisode_href,show_dvdCountry_name,show_dvdCountry_code,show_dvdCountry_timezone
0,1979824,https://www.tvmaze.com/episodes/1979824/sim-fo...,Chanyeol's Episode 16,4,16.0,regular,2020-12-01,06:00,2020-11-30T21:00:00+00:00,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,...,308.0,ТНТ,Russian Federation,RU,Asia/Kamchatka,NaN,NaN,NaN,NaN,NaN
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,https://api.tvmaze.com/episodes/2309427,NaN,NaN,NaN


Procedemos a reemplazar los valores NaN por 0 en las columns de tipo float.

In [14]:
df['runtime'] = df['runtime'].fillna(0)
df['rating_average'] = df['rating_average'].fillna(0)
df['number'] = df['number'].fillna(0)
df['show_runtime'] = df['show_runtime'].fillna(0)
df['show_averageRuntime'] = df['show_averageRuntime'].fillna(0)
df['show_rating_average'] = df['show_rating_average'].fillna(0)
df['show_externals_thetvdb'] = df['show_externals_thetvdb'].fillna(0)
df['show_network_id'] = df['show_network_id'].fillna(0)
df['show_webChannel_id'] = df['show_webChannel_id'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

A continuación pasaremos la lista de géneros a filas.
Antes tenemos que dividir el dataframe en dos dataframes: 
1. dataframe con los registros cuya lista de la columna show_genres está vacía
2. dataframe con los registros cuya lista tiene al menos un género.

Al segundo dataframe aplicaremos la función para obtener un género por fila

In [15]:
df['len_show_genres'] = df['show_genres'].str.len()
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,...,show_network_name,show_network_country_name,show_network_country_code,show_network_country_timezone,show_network_officialSite,show__links_nextepisode_href,show_dvdCountry_name,show_dvdCountry_code,show_dvdCountry_timezone,len_show_genres
0,1979824,https://www.tvmaze.com/episodes/1979824/sim-fo...,Chanyeol's Episode 16,4,16.0,regular,2020-12-01,06:00,2020-11-30T21:00:00+00:00,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,...,ТНТ,Russian Federation,RU,Asia/Kamchatka,NaN,NaN,NaN,NaN,NaN,1
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8.0,...,NaN,NaN,NaN,NaN,NaN,https://api.tvmaze.com/episodes/2309427,NaN,NaN,NaN,4


In [16]:
df_0 = df[df['len_show_genres']==0]
df_1 = df[df['len_show_genres']!=0]

In [17]:
print(df.shape)
print(df_0.shape)
print(df_1.shape)

(3037, 57)
(745, 57)
(2292, 57)


In [18]:
df_1_mod = df_1.explode('show_genres')
df_1_mod.head()

,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,...,show_network_name,show_network_country_name,show_network_country_code,show_network_country_timezone,show_network_officialSite,show__links_nextepisode_href,show_dvdCountry_name,show_dvdCountry_code,show_dvdCountry_timezone,len_show_genres
1,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,...,ТНТ,Russian Federation,RU,Asia/Kamchatka,NaN,NaN,NaN,NaN,NaN,1
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


Validamos que df_1_mod tenga las mismas columnas para hacer el concat y volver a nuestro dataframe que cargaremos a la base de datos

In [19]:
df_1_mod.shape

(4932, 57)

In [20]:
df_0['show_genres'] = df_0['show_genres'].str.strip('[]')
df_0.dtypes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


id                                    int64
url                                  object
name                                 object
season                                int64
number                              float64
type                                 object
airdate                              object
airtime                              object
airstamp                             object
runtime                             float64
summary                              object
rating_average                      float64
_links_self_href                     object
show_id                               int64
show_url                             object
show_name                            object
show_type                            object
show_language                        object
show_genres                         float64
show_status                          object
show_runtime                        float64
show_averageRuntime                 float64
show_premiered                  

In [21]:
df_0['show_genres'] = df_0['show_genres'].astype(str)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_final = pd.concat([df_1_mod, df_0], ignore_index=True)
df_final.head()

,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,...,show_network_name,show_network_country_name,show_network_country_code,show_network_country_timezone,show_network_officialSite,show__links_nextepisode_href,show_dvdCountry_name,show_dvdCountry_code,show_dvdCountry_timezone,len_show_genres
0,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,...,ТНТ,Russian Federation,RU,Asia/Kamchatka,NaN,NaN,NaN,NaN,NaN,1
2,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [23]:
df_final.drop('len_show_genres', axis=1, inplace=True)
df_final.head()

,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,...,show_network_id,show_network_name,show_network_country_name,show_network_country_code,show_network_country_timezone,show_network_officialSite,show__links_nextepisode_href,show_dvdCountry_name,show_dvdCountry_code,show_dvdCountry_timezone
0,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,...,308.0,ТНТ,Russian Federation,RU,Asia/Kamchatka,NaN,NaN,NaN,NaN,NaN
2,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5677 entries, 0 to 5676
Data columns (total 56 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                5677 non-null   int64  
 1   url                               5677 non-null   object 
 2   name                              5677 non-null   object 
 3   season                            5677 non-null   int64  
 4   number                            5677 non-null   float64
 5   type                              5677 non-null   object 
 6   airdate                           5677 non-null   object 
 7   airtime                           5677 non-null   object 
 8   airstamp                          5677 non-null   object 
 9   runtime                           5677 non-null   float64
 10  summary                           1734 non-null   object 
 11  rating_average                    5677 non-null   float64
 12  _links

Almacenaremos el dataframe final en una base de datos.

In [25]:
engine = create_engine('sqlite:///lulo_data.db', echo=True)
sqlite_connection = engine.connect()

Se eliminan también las columnas summary, show_summary y show_schedule_day dado que el contenido de los registros contiene caracteres o tiene un estructura que impide la carga del dataframe a la DB. Ademas, no serán necesarios para responder las preguntas de negocio.

In [26]:
df_final.drop(['summary', 'show_summary','show_schedule_days'], axis=1, inplace=True)

In [27]:
sqlite_table = "Series_tv_maze"
df_final.to_sql(sqlite_table, sqlite_connection, if_exists='fail', index=False)

2022-07-15 03:12:41,697 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Series_tv_maze")
2022-07-15 03:12:41,700 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:12:41,705 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Series_tv_maze")
2022-07-15 03:12:41,708 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:12:41,725 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Series_tv_maze" (
	id BIGINT, 
	url TEXT, 
	name TEXT, 
	season BIGINT, 
	number FLOAT, 
	type TEXT, 
	airdate TEXT, 
	airtime TEXT, 
	airstamp TEXT, 
	runtime FLOAT, 
	rating_average FLOAT, 
	_links_self_href TEXT, 
	show_id BIGINT, 
	show_url TEXT, 
	show_name TEXT, 
	show_type TEXT, 
	show_language TEXT, 
	show_genres TEXT, 
	show_status TEXT, 
	show_runtime FLOAT, 
	"show_averageRuntime" FLOAT, 
	show_premiered TEXT, 
	show_ended TEXT, 
	"show_officialSite" TEXT, 
	show_schedule_time TEXT, 
	show_rating_average FLOAT, 
	show_weight BIGINT, 
	"show_webChannel_id" FLOAT, 
	"show_webChannel_name" TE

Probamos que cargamos la tabla correctamente

In [28]:
test = pd.read_sql('SELECT * FROM Series_tv_maze LIMIT 10', sqlite_connection)
test

2022-07-15 03:12:49,097 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM Series_tv_maze LIMIT 10")
2022-07-15 03:12:49,101 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:12:49,103 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM Series_tv_maze LIMIT 10")
2022-07-15 03:12:49,107 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:12:49,110 INFO sqlalchemy.engine.Engine SELECT * FROM Series_tv_maze LIMIT 10
2022-07-15 03:12:49,113 INFO sqlalchemy.engine.Engine [raw sql] ()


,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,...,show_network_id,show_network_name,show_network_country_name,show_network_country_code,show_network_country_timezone,show_network_officialSite,show__links_nextepisode_href,show_dvdCountry_name,show_dvdCountry_code,show_dvdCountry_timezone
0,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,...,0.0,None,None,None,None,None,None,None,None,None
1,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,...,308.0,ТНТ,Russian Federation,RU,Asia/Kamchatka,None,None,None,None,None
2,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,0.0,None,None,None,None,None,None,None,None,None
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,0.0,None,None,None,None,None,None,None,None,None
4,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,...,0.0,None,None,None,None,None,None,None,None,None
5,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8.0,...,0.0,None,None,None,None,None,https://api.tvmaze.com/episodes/2309427,None,None,None
6,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8.0,...,0.0,None,None,None,None,None,https://api.tvmaze.com/episodes/2309427,None,None,None
7,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8.0,...,0.0,None,None,None,None,None,https://api.tvmaze.com/episodes/2309427,None,None,None
8,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8.0,...,0.0,None,None,None,None,None,https://api.tvmaze.com/episodes/2309427,None,None,None
9,2315116,https://www.tvmaze.com/episodes/2315116/sono-k...,Episode 5,1,5.0,regular,2020-12-01,,2020-12-01T03:00:00+00:00,15.0,...,0.0,None,None,None,None,None,None,None,None,None


Se reponden a las preguntas de negocio

AverageRuntime por type y mes

In [29]:
Q1 = pd.read_sql("SELECT show_type, AVG(show_averageRuntime) as avg_runtime FROM Series_tv_maze GROUP BY 1 ORDER BY 2 DESC", sqlite_connection)
Q1

2022-07-15 03:13:25,212 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT show_type, AVG(show_averageRuntime) as avg_runtime FROM Series_tv_maze GROUP BY 1 ORDER BY 2 DESC")
2022-07-15 03:13:25,218 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:13:25,222 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT show_type, AVG(show_averageRuntime) as avg_runtime FROM Series_tv_maze GROUP BY 1 ORDER BY 2 DESC")
2022-07-15 03:13:25,224 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:13:25,226 INFO sqlalchemy.engine.Engine SELECT show_type, AVG(show_averageRuntime) as avg_runtime FROM Series_tv_maze GROUP BY 1 ORDER BY 2 DESC
2022-07-15 03:13:25,227 INFO sqlalchemy.engine.Engine [raw sql] ()


,show_type,avg_runtime
0,Award Show,218.000000
1,News,94.393443
2,Sports,89.974684
3,Game Show,60.803030
4,Panel Show,45.000000
5,Talk Show,38.821705
6,Reality,36.252747
7,Scripted,36.136620
8,Documentary,28.360976
9,Variety,26.138614


Cantidad de series emitidas por género por mes

In [30]:
Q2 = pd.read_sql("SELECT show_genres AS genre, COUNT(DISTINCT show_id) AS quantity_shows FROM Series_tv_maze WHERE show_genres != 'nan' GROUP BY 1 ORDER BY 2 DESC", sqlite_connection)
Q2

2022-07-15 03:13:28,341 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT show_genres AS genre, COUNT(DISTINCT show_id) AS quantity_shows FROM Series_tv_maze WHERE show_genres != 'nan' GROUP BY 1 ORDER BY 2 DESC")
2022-07-15 03:13:28,343 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:13:28,345 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT show_genres AS genre, COUNT(DISTINCT show_id) AS quantity_shows FROM Series_tv_maze WHERE show_genres != 'nan' GROUP BY 1 ORDER BY 2 DESC")
2022-07-15 03:13:28,347 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:13:28,352 INFO sqlalchemy.engine.Engine SELECT show_genres AS genre, COUNT(DISTINCT show_id) AS quantity_shows FROM Series_tv_maze WHERE show_genres != 'nan' GROUP BY 1 ORDER BY 2 DESC
2022-07-15 03:13:28,353 INFO sqlalchemy.engine.Engine [raw sql] ()


,genre,quantity_shows
0,Comedy,180
1,Drama,169
2,Romance,89
3,Children,40
4,Crime,40
5,Action,38
6,Adventure,35
7,Fantasy,35
8,Mystery,31
9,Anime,30


Porcentaje por país respecto al total de series emitidas por mes.

In [31]:
Q3 = pd.read_sql("SELECT show_webChannel_country_name AS country, CAST(COUNT(DISTINCT show_id) AS FLOAT)/CAST((SELECT COUNT(DISTINCT show_id) FROM Series_tv_maze WHERE show_webChannel_country_name IS NOT NULL) AS FLOAT) AS quantity_shows FROM Series_tv_maze WHERE show_webChannel_country_name IS NOT NULL GROUP BY 1 ORDER BY 2 DESC", sqlite_connection)
Q3

2022-07-15 03:13:47,724 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT show_webChannel_country_name AS country, CAST(COUNT(DISTINCT show_id) AS FLOAT)/CAST((SELECT COUNT(DISTINCT show_id) FROM Series_tv_maze WHERE show_webChannel_country_name IS NOT NULL) AS FLOAT) AS quantity_shows FROM Series_tv_maze WHERE show_webChannel_country_name IS NOT NULL GROUP BY 1 ORDER BY 2 DESC")
2022-07-15 03:13:47,729 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:13:47,735 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT show_webChannel_country_name AS country, CAST(COUNT(DISTINCT show_id) AS FLOAT)/CAST((SELECT COUNT(DISTINCT show_id) FROM Series_tv_maze WHERE show_webChannel_country_name IS NOT NULL) AS FLOAT) AS quantity_shows FROM Series_tv_maze WHERE show_webChannel_country_name IS NOT NULL GROUP BY 1 ORDER BY 2 DESC")
2022-07-15 03:13:47,738 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:13:47,742 INFO sqlalchemy.engine.Engine SELECT show_webChannel_c

,country,quantity_shows
0,United States,0.228374
1,China,0.169550
2,Norway,0.124567
3,Russian Federation,0.096886
4,"Korea, Republic of",0.079585
5,United Kingdom,0.041522
6,Turkey,0.031142
7,Germany,0.024221
8,India,0.024221
9,Japan,0.020761


Rating promedio de series por país y género

In [32]:
Q4 = pd.read_sql("SELECT show_genres AS genre, show_webChannel_country_name AS country, AVG(show_rating_average) AS avg_rating FROM Series_tv_maze WHERE show_genres != 'nan' AND show_webChannel_country_name IS NOT NULL GROUP BY 1,2 HAVING AVG(show_rating_average)>0 ORDER BY 3 DESC", sqlite_connection)
Q4

2022-07-15 03:13:53,544 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT show_genres AS genre, show_webChannel_country_name AS country, AVG(show_rating_average) AS avg_rating FROM Series_tv_maze WHERE show_genres != 'nan' AND show_webChannel_country_name IS NOT NULL GROUP BY 1,2 HAVING AVG(show_rating_average)>0 ORDER BY 3 DESC")
2022-07-15 03:13:53,546 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:13:53,549 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT show_genres AS genre, show_webChannel_country_name AS country, AVG(show_rating_average) AS avg_rating FROM Series_tv_maze WHERE show_genres != 'nan' AND show_webChannel_country_name IS NOT NULL GROUP BY 1,2 HAVING AVG(show_rating_average)>0 ORDER BY 3 DESC")
2022-07-15 03:13:53,553 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-15 03:13:53,555 INFO sqlalchemy.engine.Engine SELECT show_genres AS genre, show_webChannel_country_name AS country, AVG(show_rating_average) AS avg_rating FROM Series_tv_

,genre,country,avg_rating
0,History,Norway,8.000000
1,War,Norway,8.000000
2,Drama,Philippines,7.800000
3,Romance,Philippines,7.800000
4,Anime,United States,7.700000
5,Science-Fiction,United States,7.700000
6,Supernatural,United Kingdom,7.600000
7,Drama,United Kingdom,7.100000
8,Thriller,United Kingdom,7.100000
9,Supernatural,United States,7.100000


Cerramos la conexión luego de cargar el df a la tabla, probar que tenemos los datos ejecutando una consulta sencilla de SQL y responder a las preguntas de negocio.

In [33]:
sqlite_connection.close()